source : https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb#scrollTo=BZSlp3DAjdYf

In [20]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

This tutorial is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. To run all the code in the notebook, select **Runtime** > **Run all**. To run the code cells one at a time, hover over each cell and select the **Run cell** icon.

![Run cell icon](images/beginner/run_cell_icon.png)

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [22]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.6.3


If you are following along in your own development environment, rather than [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb), see the [install guide](https://www.tensorflow.org/install) for setting up TensorFlow for development.

Note: Make sure you have upgraded to the latest `pip` to install the TensorFlow 2 package if you are using your own development environment. See the [install guide](https://www.tensorflow.org/install) for details.

## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by `255.0`. This also converts the sample data from integers to floating-point numbers:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 34s 3us/step


## Build a machine learning model

Build a `tf.keras.Sequential` model:

In [24]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [10]:
x_train.shape

(60000, 28, 28)

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.05673968,  0.18218818, -0.56821275, -1.1317579 , -0.3621098 ,
        -0.6768396 ,  0.42543244,  0.21999116,  0.07571939, -0.6232739 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class: 

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.12105717, 0.13723727, 0.06480031, 0.03688354, 0.0796318 ,
        0.0581301 , 0.1750297 , 0.14252457, 0.12337673, 0.06132879]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. 

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.8450718

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [25]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [27]:
model.fit(x_train, y_train, epochs=15)

Epoch 1/15
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0658 - accuracy: 0.9794
Epoch 2/15
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0584 - accuracy: 0.9814
Epoch 3/15
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0536 - accuracy: 0.9827
Epoch 4/15
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0490 - accuracy: 0.9843
Epoch 5/15
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0457 - accuracy: 0.9850
Epoch 6/15
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0432 - accuracy: 0.9858
Epoch 7/15
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0382 - accuracy: 0.9872
Epoch 8/15
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0365 - accuracy: 0.9878
Epoch 9/15
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0345 - accuracy: 0.9883
Epoch 10/15
1875/1875 [==============================] - 7s 4ms/step - loss: 0.032

The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [28]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0751 - accuracy: 0.9818


[0.07511146366596222, 0.9818000197410583]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [30]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [31]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[7.94075986e-11, 1.47083373e-10, 3.83711847e-11, 9.89540877e-06,
        1.04364369e-14, 2.27142916e-10, 5.47659417e-20, 9.99989271e-01,
        1.23549015e-09, 8.11112784e-07],
       [3.31650058e-14, 2.32981012e-08, 1.00000000e+00, 2.01275063e-09,
        1.58805356e-30, 5.93542576e-12, 3.89753941e-12, 2.00587779e-25,
        5.22586497e-14, 2.55177496e-25],
       [3.28074970e-13, 9.99996901e-01, 2.60840451e-08, 1.07164488e-09,
        1.72180901e-08, 1.41044051e-10, 3.30100747e-08, 4.29330555e-07,
        2.46939726e-06, 1.22881968e-10],
       [9.99998450e-01, 2.66616671e-13, 8.21404171e-07, 3.94798994e-13,
        3.92260660e-12, 6.83460066e-10, 2.76661343e-08, 1.91936658e-11,
        1.01987773e-14, 6.72693147e-07],
       [2.98799735e-10, 3.32313535e-15, 2.11717643e-09, 7.49777031e-13,
        9.99878049e-01, 5.30442313e-15, 2.91091066e-12, 2.68083613e-06,
        1.78850088e-10, 1.19339325e-04]], dtype=float32)>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).
